In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine
import datetime
import random
import re


# Fake data creation

In [2]:
from numpy import integer


fake=Faker()

fake_data = defaultdict(list)

row_number=int(input("Please write the number you wish to see the row size, DON'T CRASH ME!"))



In [3]:
# Creating sender_id
for i in range(row_number):
    fake_data['sender_id'].append(fake.bothify(text='####'))
#fake_data['receiver_id']=fake_data['sender_id']

sender_id=fake_data["sender_id"]

manipulated_receiver_id=[]# yeni data frame '  e  eklemek içni yeni to_ custormer listem 
amount=[]
timestamp=[]

Enjecting fake_data to DataFrame

In [4]:
for idx in range(len(fake_data['sender_id'])): # fake_data ['sender_id'] nin uzunluğu kadar bir döngü yapiyorum
    # rastgele random sayi üretiyorum ki from_customer ' random sayida aktivasyonu olsun
    #f_c_s=''.join(sender_id[i])
    temp_sender_val=sender_id[idx]
    temp_receiver_list=[]
    temp_amount_list=[]
    temp_timestamp_list=[]

    for j in range(random.randint(1,10)): #random sayiyla döngü kuruyorum 

        # ayni müşteri kendine göndermemesi iiçin seçimi parçaliyorum
        
        rnd_reciever_id=''.join(random.choices(sender_id))         #to_customer listesinden random seçiyorum 
        
        if (temp_sender_val== rnd_reciever_id):  # ayni müşteri kendine göndermemesi id leri  kiyasliyorum eğer ayniysa döngüye devam ediyor
            continue

        temp_receiver_list.append(rnd_reciever_id)
        temp_amount_list.append(fake.pyfloat(right_digits=2,positive=True,min_value=2.00,max_value=10000.00))
        temp_timestamp_list.append(fake.date_time_ad(start_datetime=datetime.datetime(2022,9,14)))
    
    if len(manipulated_receiver_id)==len(fake_data['sender_id']):
            break
    # random seçtiğimiz ögeyi yeni to_custoemr listesine ekliyoruz
    manipulated_receiver_id.append(temp_receiver_list)
    amount.append(temp_amount_list)
    timestamp.append(temp_timestamp_list)
     # random seçtiğimiz ögeyi yeni to_custoemr listesine ekliyoruz
        

In [5]:
fake_data['receiver_id']=manipulated_receiver_id
fake_data['amount']=amount
fake_data['timestamp']=timestamp
print(len(fake_data['amount']))
df=pd.DataFrame(fake_data)
df=df[['sender_id','receiver_id','amount','timestamp']]
df

20


,sender_id,receiver_id,amount,timestamp
0,3128,"[2805, 8786, 0158, 0791, 5093, 6880, 0791, 9390]","[8799.75, 846.41, 2858.98, 6744.3, 726.83, 600...","[2022-10-09 04:28:12, 2022-10-17 14:19:17, 202..."
1,9119,"[6220, 7934, 5093, 4377, 8786, 9390]","[5407.13, 7390.36, 5294.65, 7416.86, 7983.23, ...","[2022-10-15 04:53:01, 2022-10-28 02:59:58, 202..."
2,2360,"[3128, 9796, 3128, 6880, 1169, 0158, 6220]","[6184.73, 1348.47, 7898.12, 250.61, 4211.53, 1...","[2022-09-25 14:10:06, 2022-09-28 22:35:51, 202..."
3,7868,"[9390, 0791]","[5059.1, 2766.49]","[2022-10-28 01:11:49, 2022-10-08 22:11:35]"
4,1169,"[2360, 6005]","[7041.23, 6003.5]","[2022-10-19 23:26:28, 2022-09-18 22:24:24]"
5,0791,"[6220, 9119, 7868, 4377, 1169, 6005, 8786]","[797.56, 2028.71, 3828.55, 6383.83, 8479.9, 27...","[2022-10-05 21:22:11, 2022-10-24 20:07:57, 202..."
6,5320,"[7934, 9796]","[8748.89, 9456.25]","[2022-10-30 04:46:45, 2022-10-10 22:01:05]"
7,2805,"[9390, 6220, 1169, 6880, 6182, 7868, 9796, 7868]","[4740.47, 1490.69, 4373.16, 7612.83, 3542.28, ...","[2022-09-19 08:32:42, 2022-10-22 09:12:03, 202..."
8,7934,"[4377, 4377, 6220, 3759, 9390, 4377, 6005]","[4617.33, 4748.74, 637.4, 1049.18, 3288.28, 48...","[2022-10-26 19:06:42, 2022-09-18 20:49:09, 202..."
9,8786,"[0158, 6182]","[1857.84, 1595.99]","[2022-09-23 06:31:26, 2022-10-07 18:11:03]"


# Neo4j

In [6]:
from neo4j import GraphDatabase

In [7]:
transaction_info=df.values.tolist()

In [8]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "test"))

def clear_db(tx):
    tx.run("MATCH(n) Detach Delete n")

def create_customer(tx, customer_id):
    tx.run("CREATE (c1:customer {name: $name, customer_id: $id})",name='customer', id=customer_id)

def create_transaction(tx,customer_id,other_customer_id,amount,timestamp):
    tx.run("MATCH (c1:customer),(c2:customer) "
    "Where c1.customer_id= $id and c2.customer_id= $otherid "
    "Create (c1)-[t:Transaction{amount: $amount, time: $timestamp}]->(c2)",
    id=customer_id,otherid=other_customer_id,amount=amount,timestamp=timestamp)


with driver.session() as session:

    session.execute_write(clear_db)

    for i in range(len(transaction_info)):
        session.write_transaction(create_customer, str(transaction_info[i][0]))

    
    for element in transaction_info:
        for block in range(1,len(element)):
            for j in range(len(element[block])):

                session.write_transaction(create_transaction,element[0],element[1][j],element[2][j],element[3][j])

driver.close()

Unable to retrieve routing information
Transaction failed and will be retried in 1.1774183650869239s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 1.6572200177458045s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 4.539629179626631s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 6.717906573221589s (Unable to retrieve routing information)


KeyboardInterrupt: 